### 預先載入要用的資料

In [2]:
import numpy as np
import pandas as pd
import math as m

country = pd.read_csv('country.csv', encoding='Big5')
league = pd.read_csv('league.csv', encoding='Big5')
match = pd.read_csv('match.csv', encoding='Big5')
player = pd.read_csv('player.csv', encoding='Big5')
player_attribute = pd.read_csv('player_attribute.csv' ,encoding='Big5')
position = pd.read_csv('player_positions.csv', encoding='Big5')

### League
與 Country 合併

In [3]:
# 合併
league_merge = pd.merge(league, country, how='inner', on='id')

# 移除無用column
league_merge.drop(['id', 'country_id'], axis=1)
league_merge.drop('country_id', inplace=True, axis=1)

# column name重新命名
league_merge.rename(columns={
    'id':'league_id',
    'name_x':'name',
    'name_y':'name_simple'
}, inplace = True)

print(league_merge)

    league_id                      name  name_simple
0           1    Belgium Jupiler League      Belgium
1        1729    England Premier League      England
2        4769            France Ligue 1       France
3        7809     Germany 1. Bundesliga      Germany
4       10257             Italy Serie A        Italy
5       13274    Netherlands Eredivisie  Netherlands
6       15722        Poland Ekstraklasa       Poland
7       17642  Portugal Liga ZON Sagres     Portugal
8       19694   Scotland Premier League     Scotland
9       21518           Spain LIGA BBVA        Spain
10      24558  Switzerland Super League  Switzerland


### Match
確認可用筆數 + 清理match資料

In [4]:
# 建subset: 需要篩缺值的column(該場比賽的選手名單)
match_subset = match[['id', 'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']]

# 把上場球員資料有缺值的篩掉
match_subset_cleaned = match_subset.dropna()

# 把原始資料與subset合併
match_all = pd.merge(match, match_subset_cleaned, how="inner")

# 清理無用column
match_all = match_all.iloc[:, 0:77]
match_all.drop('id', axis=1, inplace=True)

print(match_all)

       country_id  league_id     season  stage                 date  \
0               1          1  2008/2009     24  2009-02-27 00:00:00   
1               1          1  2008/2009     25  2009-03-08 00:00:00   
2               1          1  2008/2009     25  2009-03-07 00:00:00   
3               1          1  2008/2009     26  2009-03-13 00:00:00   
4               1          1  2008/2009     26  2009-03-14 00:00:00   
...           ...        ...        ...    ...                  ...   
21369       24558      24558  2015/2016      8  2015-09-13 00:00:00   
21370       24558      24558  2015/2016      9  2015-09-22 00:00:00   
21371       24558      24558  2015/2016      9  2015-09-23 00:00:00   
21372       24558      24558  2015/2016      9  2015-09-23 00:00:00   
21373       24558      24558  2015/2016      9  2015-09-23 00:00:00   

       match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0            493017              8203              9987               2 

### Player

#### 星座

In [5]:
# 把月跟日切出來
splitted = player['birthday'].apply(lambda x : x.split('/'))
# 月
month = list(splitted.apply(lambda x : int(x[1])))
player['birthday_month'] = month
# 日
day = list(splitted.apply(lambda x : int(x[2].split(' ')[0])))
player['birthday_day'] = day

# 判斷星座
def which_star_sign(month, day):
    if (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Aries"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Taurus"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Gemini"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Cancer"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Leo"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Virgo"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Libra"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Scopio"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Sagittarius"
    elif (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Capricorn"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Aquarius"
    else:
        return "Pisces"
    
player['star_sign'] = list(map(lambda x, y :which_star_sign(x, y),player['birthday_month'], player['birthday_day']))

# 清掉不需要的column
player.drop('id', inplace=True, axis=1)
player.drop('birthday_month', inplace=True, axis=1)
player.drop('birthday_day', inplace=True, axis=1)

print(player)


       player_api_id          player_name  player_fifa_api_id  \
0             505942   Aaron Appindangoye              218353   
1             155782      Aaron Cresswell              189615   
2             162549          Aaron Doran              186170   
3              30572        Aaron Galindo              140161   
4              23780         Aaron Hughes               17725   
...              ...                  ...                 ...   
11055          26357       Zoumana Camara                2488   
11056         111182         Zsolt Laczko              164680   
11057          36491            Zsolt Low              111191   
11058          35506  Zurab Khizanishvili               47058   
11059          39902   Zvjezdan Misimovic              102359   

               birthday  height  weight    star_sign  
0       1992/2/29 00:00  182.88     187       Pisces  
1      1989/12/15 00:00  170.18     146  Sagittarius  
2       1991/5/13 00:00  170.18     163       Taurus  

#### Player attribute 清理 
        

Why. 一個球員有多筆attribute紀錄(不同時間點的)  

How. 先清missing 後整理  

Missing 資料處理方法:  

        Case1 全部missing: None
        Case2 部分missing: drop該筆 
球員素質整理:  

        Categorical data: 取最新的一筆data    
        Numeric data: 取平均

In [6]:
# 清掉無用column
player_attribute.drop('player_fifa_api_id', axis=1, inplace=True)

# 所有player id
player_id = player['player_api_id']

# 建List: 存每個球員的素質資料，用來建dataframe用
id = list()
player_api_id = list()
overall_rating = list()
potential = list()
crossing = list()
finishing = list()
heading_accuracy = list()
short_passing = list()
volleys = list()
dribbling = list()
curve = list()
free_kick_accuracy = list()
long_passing = list()
ball_control = list()
acceleration = list()
sprint_speed = list()
agility = list()
reactions = list()
balance = list()
shot_power = list()
jumping = list()
stamina = list()
strength = list()
long_shots = list()
aggression = list()
interceptions = list()
positioning = list()
vision = list()
penalties = list()
marking = list()
standing_tackle = list()
sliding_tackle = list()
gk_diving = list()
gk_handling = list()
gk_kicking = list()
gk_positioning = list()
gk_reflexes = list()
preferred_foot= list()
attacking_work_rate = list()
defensive_work_rate = list()


# 迴圈存資料環節
for i in player_id:
    # 找出屬於每個球員的資料，建成Dictionary(簡稱dic)
    player_attribute_id = player_attribute['player_api_id']    
    temp_dic = player_attribute[player_attribute_id == i]

    # 每個attribute依序處理
    for j in player_attribute.columns:
        # 複製一份該球員的dic(用來計算)並移除該attribute中的空值
        temp = temp_dic[j].copy()
        temp.dropna(inplace=True)
        match j:
            # id相關
            case 'player_api_id':
                player_api_id = player_api_id + [i]
            # Numerical data: 平均
            case 'overall_rating':
                overall_rating = overall_rating + [np.mean(temp)]
            case 'potential':
                potential = potential + [np.mean(temp)]
            case 'crossing':
                crossing = crossing + [np.mean(temp)]
            case 'finishing':
                finishing = finishing + [np.mean(temp)]
            case 'heading_accuracy':
                heading_accuracy = heading_accuracy + [np.mean(temp)]
            case 'short_passing':
                short_passing = short_passing + [np.mean(temp)]
            case 'volleys':
                volleys = volleys + [np.mean(temp)]
            case 'dribbling':
                dribbling = dribbling + [np.mean(temp)]
            case 'curve':
                curve = curve + [np.mean(temp)]
            case 'free_kick_accuracy':
                free_kick_accuracy = free_kick_accuracy + [np.mean(temp)]
            case 'long_passing':
                long_passing = long_passing + [np.mean(temp)]
            case 'ball_control':
                ball_control = ball_control + [np.mean(temp)]
            case 'acceleration':
                acceleration = acceleration + [np.mean(temp)]
            case 'sprint_speed':
                sprint_speed = sprint_speed + [np.mean(temp)]
            case 'agility':
                agility = agility + [np.mean(temp)]
            case 'reactions':
                reactions = reactions + [np.mean(temp)]
            case 'balance':
                balance = balance + [np.mean(temp)]
            case 'shot_power':
                shot_power = shot_power + [np.mean(temp)]
            case 'jumping':
                jumping = jumping + [np.mean(temp)]
            case 'stamina':
                stamina = stamina + [np.mean(temp)]
            case 'strength':
                strength = strength + [np.mean(temp)]
            case 'long_shots':
                long_shots = long_shots + [np.mean(temp)]
            case 'aggression':
                aggression = aggression + [np.mean(temp)]
            case 'interceptions':
                interceptions = interceptions + [np.mean(temp)]
            case 'positioning':
                positioning = positioning + [np.mean(temp)]
            case 'vision':
                vision = vision + [np.mean(temp)]
            case 'penalties':
                penalties = penalties + [np.mean(temp)]
            case 'marking':
                marking = marking + [np.mean(temp)]
            case 'standing_tackle':
                standing_tackle = standing_tackle + [np.mean(temp)]
            case 'sliding_tackle':
                sliding_tackle = sliding_tackle + [np.mean(temp)]
            case 'gk_diving':
                gk_diving = gk_diving + [np.mean(temp)]
            case 'gk_handling':
                gk_handling = gk_handling + [np.mean(temp)]
            case 'gk_kicking':
                gk_kicking = gk_kicking + [np.mean(temp)]
            case 'gk_positioning':
                gk_positioning = gk_positioning + [np.mean(temp)]
            case 'gk_reflexes':
                gk_reflexes = gk_reflexes + [np.mean(temp)]
            # Categorical data: 取最新的一筆，沒資料(全是空值)就填None
            case 'preferred_foot':
                if(len(temp) > 0):
                    preferred_foot = preferred_foot + [temp.iloc[0]]
                else:
                    preferred_foot = preferred_foot + [None]
            case 'attacking_work_rate':
                if(len(temp) > 0):
                    attacking_work_rate = attacking_work_rate + [temp.iloc[0]]
                else:
                    attacking_work_rate = attacking_work_rate + [None]
            case 'defensive_work_rate':
                if(len(temp) > 0):
                    defensive_work_rate = defensive_work_rate + [temp.iloc[0]]
                else:
                    defensive_work_rate = defensive_work_rate + [None]

# 建Dataframe
df = pd.DataFrame({
    'player_api_id' : player_api_id,
    'overall_rating' : overall_rating,
    'potential' : potential,
    'crossing' : crossing,
    'finishing' : finishing,
    'heading_accuracy' : heading_accuracy,
    'short_passing' : short_passing,
    'volleys' : volleys,
    'dribbling' : dribbling,
    'curve' : curve,
    'free_kick_accuracy' : free_kick_accuracy,
    'long_passing' : long_passing,
    'ball_control' : ball_control,
    'acceleration' : acceleration,
    'sprint_speed' : sprint_speed,
    'agility' : agility,
    'reactions' : reactions,
    'balance' : balance,
    'shot_power' : shot_power,
    'jumping' : jumping,
    'stamina' : stamina,
    'strength' : strength,
    'long_shots' : long_shots,
    'aggression' : aggression,
    'interceptions' : interceptions,
    'positioning' : positioning,
    'vision' : vision,
    'penalties' : penalties,
    'marking' : marking,
    'standing_tackle' : standing_tackle, 
    'sliding_tackle' : sliding_tackle,
    'gk_diving' : gk_diving,
    'gk_handling' : gk_handling,
    'gk_kicking' : gk_kicking,
    'gk_positioning' : gk_positioning,
    'gk_reflexes' : gk_reflexes,
    'preferred_foot' : preferred_foot,
    'attacking_work_rate' : attacking_work_rate,
    'defensive_work_rate' : defensive_work_rate
}) 
    
print(df)   

       player_api_id  overall_rating  potential   crossing  finishing  \
0             505942       63.600000  67.600000  48.600000  43.600000   
1             155782       66.969697  74.484848  70.787879  49.454545   
2             162549       67.000000  74.192308  68.115385  57.923077   
3              30572       69.086957  70.782609  57.217391  26.260870   
4              23780       73.240000  74.680000  45.080000  38.840000   
...              ...             ...        ...        ...        ...   
11055          26357       74.384615  75.461538  42.000000  27.000000   
11056         111182       65.687500  71.625000  67.250000  46.750000   
11057          36491       67.571429  72.857143  63.142857  44.571429   
11058          35506       70.750000  78.125000  46.750000  43.000000   
11059          39902       80.000000  81.700000  78.200000  72.600000   

       heading_accuracy  short_passing    volleys  dribbling      curve  ...  \
0             70.600000      60.600000  43.

#### 處理Position

In [7]:
position['Position'] = (position[(position == 1).sum(axis =1)==1]==1).idxmax(axis =1)
position['Position'] = position['Position'].fillna("None")
print(position)

       playerID  CAM  CB  CDM  CF  CM  GK  LB  LF  LM  LW  LWB  RB  RF  RM  \
0             2    0   0    0   0   0   0   0   0   1   0    0   0   0   0   
1             6    0   0    0   0   0   1   0   0   0   0    0   0   0   0   
2            11    0   0    2   0   1   0   0   0   0   0    0   0   0   0   
3            16    2   0    0   0   1   0   0   0   0   0    0   0   0   0   
4            27    1   0    0   0   0   0   0   0   4   0    0   0   0   2   
...         ...  ...  ..  ...  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..   
11055    233885    0   1    0   0   0   0   0   0   0   0    0   0   0   0   
11056    233911    0   0    0   0   0   0   1   0   0   0    2   0   0   0   
11057    233930    0   0    0   0   0   0   0   0   0   0    0   1   0   0   
11058    233969    0   2    0   0   0   0   0   0   0   0    0   1   0   0   
11059    234141    0   0    1   0   2   0   0   0   0   0    0   0   0   0   

       RW  RWB  ST  SW Position  
0       0    0   0   0       

#### 合併
Merge player/ player_attribute/ position and 四捨五入

In [8]:
# 合併Dataframe
player_cleaned = pd.merge(left = player, right = df, on="player_api_id")
player_all = pd.merge(right=player_cleaned, left = position, how="inner", left_on="playerID", right_on="player_fifa_api_id")
# 移除無用column
player_all.drop(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LF', 'LM', 'LW', 'LWB', 'RB', 'RF', 'RM', 'RW',	'RWB', 'ST', 'SW', 'playerID'], inplace=True, axis=1)

# Numerical data四捨五入
numeric_attribute = ['height','weight','overall_rating','potential','crossing','finishing','heading_accuracy','short_passing','volleys','dribbling','curve','free_kick_accuracy','long_passing','ball_control','acceleration','sprint_speed','agility','reactions','balance','shot_power','jumping','stamina','strength','long_shots','aggression','interceptions','positioning','vision','penalties','marking','standing_tackle','sliding_tackle','gk_diving','gk_handling','gk_kicking','gk_positioning','gk_reflexes']
player_all[numeric_attribute] = round(player_all[numeric_attribute])

print(player_all)

      Position  player_api_id        player_name  player_fifa_api_id  \
0           LM          39357  Giovanni Pasquale                   2   
1           GK          41762     Alessio Scarpi                   6   
2           CM          26028      Romain Rocchi                  11   
3           CM          24852        Luis Garcia                  16   
4          CAM          30630           Joe Cole                  27   
...        ...            ...                ...                 ...   
11055       CB         705484        Eray Cumart              233885   
11056       LB         674492         Jose Matos              233911   
11057       RB         746419   Francisco Afonso              233930   
11058       RB         748432     Fynn Arkenberg              233969   
11059      CDM         750584  Alexandre Azevedo              234141   

              birthday  height  weight  star_sign  overall_rating  potential  \
0       1982/1/5 00:00   183.0     181  Capricorn      

### 匯出成CSV

In [8]:
league_merge.to_csv('league_cleaned.csv', index=False)
player_all.to_csv('player_cleaned.csv', index=False)
match_all.to_csv('match_cleaned.csv', index=False)